# 3-1,低阶API示范

下面的范例使用TensorFlow的低阶API实现线性回归模型。

低阶API主要包括张量操作，计算图和自动微分。

In [1]:
import tensorflow as tf

#打印时间分割线
@tf.function
def printbar():
    ts = tf.timestamp()
    today_ts = ts%(24*60*60)

    hour = tf.cast(today_ts//3600+8,tf.int32)%tf.constant(24)
    minite = tf.cast((today_ts%3600)//60,tf.int32)
    second = tf.cast(tf.floor(today_ts%60),tf.int32)
    
    def timeformat(m):
        if tf.strings.length(tf.strings.format("{}",m))==1:
            return(tf.strings.format("0{}",m))
        else:
            return(tf.strings.format("{}",m))
    
    timestring = tf.strings.join([timeformat(hour),timeformat(minite),
                timeformat(second)],separator = ":")
    tf.print("=========="*8,end = "")
    tf.print(timestring)

In [2]:
# 样本数量
n =400

# 生成测试用数据集
X = tf.random.uniform([n, 2], minval=-10, maxval=10)
w0 = tf.constant([[2.0], [-1.0]])
b0 = tf.constant(3.0)
Y = X@w0 + b0 + tf.random.normal([n, 1], mean=0.0, stddev=2.0)# @表示矩阵乘法,增加正态扰动

In [6]:
# 使用动态图调试

w = tf.Variable(tf.random.normal(w0.shape))
b = tf.Variable(0.0)

def train(epoches):
    for epoch in tf.range(1, epoches+1):
        with tf.GradientTape() as tape:
            # 正向传播求损失
            Y_hat = X@w + b
            loss = tf.squeeze(tf.transpose(Y-Y_hat)@(Y-Y_hat)/(2.0*n))
            # tf.squeeze 给定张量输入，此操作返回相同类型的张量，并删除所有尺寸为1的尺寸。
        # 反向传播求梯度
        dloss_dw, dloss_db = tape.gradient(loss, [w, b])
        # 梯度下降法更新参数
        w.assign(w - 0.001 * dloss_dw)
        b.assign(b - 0.001 * dloss_db)
        if epoch % 1000 == 0:
            printbar()
            tf.print("epoch =", epoch, "; loss =", loss)
            tf.print("w =", w)
            tf.print("b =", b)
            tf.print("")
train(5000)

================================================================================22:49:58
epoch = 1000 ; loss = 2.70946407
w = [[1.98515093]
 [-1.03169119]]
b = 2.02254653

================================================================================22:50:00
epoch = 2000 ; loss = 2.1017797
w = [[1.99002635]
 [-1.02539611]]
b = 2.77152085

================================================================================22:50:01
epoch = 3000 ; loss = 2.01909518
w = [[1.99182439]
 [-1.02307415]]
b = 3.04779363

================================================================================22:50:02
epoch = 4000 ; loss = 2.00784397
w = [[1.99248791]
 [-1.02221823]]
b = 3.14970684

================================================================================22:50:03
epoch = 5000 ; loss = 2.00631309
w = [[1.99273252]
 [-1.02190089]]
b = 3.18729782



In [7]:
##使用autograph机制转换成静态图加速
w = tf.Variable(tf.random.normal(w0.shape))
b = tf.Variable(0.0)

@tf.function
def train(epoches):
    for epoch in tf.range(1,epoches+1):
        with tf.GradientTape() as tape:
            #正向传播求损失
            Y_hat = X@w + b
            loss = tf.squeeze(tf.transpose(Y-Y_hat)@(Y-Y_hat))/(2.0*n)   

        # 反向传播求梯度
        dloss_dw,dloss_db = tape.gradient(loss,[w,b])
        # 梯度下降法更新参数
        w.assign(w - 0.001*dloss_dw)
        b.assign(b - 0.001*dloss_db)
        if epoch%1000 == 0:
            printbar()
            tf.print("epoch =",epoch," loss =",loss,)
            tf.print("w =",w)
            tf.print("b =",b)
            tf.print("")
train(5000)

================================================================================22:50:15
epoch = 1000  loss = 2.70483422
w = [[1.98517644]
 [-1.03165841]]
b = 2.02645826

================================================================================22:50:15
epoch = 2000  loss = 2.10114884
w = [[1.99003589]
 [-1.02538383]]
b = 2.77296615

================================================================================22:50:15
epoch = 3000  loss = 2.0190084
w = [[1.99182796]
 [-1.02306986]]
b = 3.04833055

================================================================================22:50:15
epoch = 4000  loss = 2.00783205
w = [[1.9924891]
 [-1.02221668]]
b = 3.14990139

================================================================================22:50:16
epoch = 5000  loss = 2.00631142
w = [[1.992733]
 [-1.0219003]]
b = 3.18736935

